In [19]:
import pandas as pd
import numpy as np
import psycopg2
import sqlalchemy 
from sqlalchemy import create_engine

#from config import username, password

In [20]:
movie = pd.read_csv('predictions_boxoffice.csv', index_col=0)
movie

,title,movie_rating,run_time,meta_score,imdb_rating,director,writer,actor,genre_name,Year,Predictions_BoxOffice,label_boxoffice
movie_id,,,,,,,,,,,,
1,The Shining,R,146.0,0.0,8.4,Stanley Kubrick,Stephen King,Jack Nicholson,Drama,1980,1,1
9,Superman II,PG,127.0,0.0,6.8,Richard Lester,Jerry Siegel,Gene Hackman,Action,1981,1,1
15,Dressed to Kill,R,104.0,0.0,7.1,Brian De Palma,Brian De Palma,Michael Caine,Crime,1980,1,1
16,Somewhere in Time,PG,103.0,0.0,7.2,Jeannot Szwarc,Richard Matheson,Christopher Reeve,Drama,1980,1,6
18,9 to 5,PG,109.0,0.0,6.9,Colin Higgins,Patricia Resnick,Jane Fonda,Comedy,1980,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
7881,We Need to Do Something,Unrated,97.0,48.0,4.5,Sean King O'Grady,Max Booth III,Vinessa Shaw,Horror,2021,1,0
7916,Uncharted,PG-13,116.0,42.0,6.9,Ruben Fleischer,Rafe Judkins,Tom Holland,Action,2022,1,0
7920,Moonfall,PG-13,130.0,41.0,5.3,Roland Emmerich,Spenser Cohen,Patrick Wilson,Action,2022,1,1


In [21]:
movie1 = movie.drop(['label_boxoffice'],axis=1)
movie1

,title,movie_rating,run_time,meta_score,imdb_rating,director,writer,actor,genre_name,Year,Predictions_BoxOffice
movie_id,,,,,,,,,,,
1,The Shining,R,146.0,0.0,8.4,Stanley Kubrick,Stephen King,Jack Nicholson,Drama,1980,1
9,Superman II,PG,127.0,0.0,6.8,Richard Lester,Jerry Siegel,Gene Hackman,Action,1981,1
15,Dressed to Kill,R,104.0,0.0,7.1,Brian De Palma,Brian De Palma,Michael Caine,Crime,1980,1
16,Somewhere in Time,PG,103.0,0.0,7.2,Jeannot Szwarc,Richard Matheson,Christopher Reeve,Drama,1980,1
18,9 to 5,PG,109.0,0.0,6.9,Colin Higgins,Patricia Resnick,Jane Fonda,Comedy,1980,1
...,...,...,...,...,...,...,...,...,...,...,...
7881,We Need to Do Something,Unrated,97.0,48.0,4.5,Sean King O'Grady,Max Booth III,Vinessa Shaw,Horror,2021,1
7916,Uncharted,PG-13,116.0,42.0,6.9,Ruben Fleischer,Rafe Judkins,Tom Holland,Action,2022,1
7920,Moonfall,PG-13,130.0,41.0,5.3,Roland Emmerich,Spenser Cohen,Patrick Wilson,Action,2022,1


In [22]:
upcoming = pd.read_csv('predictions_upcoming_boxoffice.csv', index_col=0)
upcoming

,title,movie_rating,run_time,meta_score,imdb_rating,director,writer,actor,genre_name,Year,Predictions_Upcoming,label_boxoffice
movie_id,,,,,,,,,,,,
1,A Day to Die,0,0,0.0,0.0,Wes Miller,Scott Mallace,Kevin Dillon,Action,1970,1,0
2,He Was Asking for It: A Noir Film,0,0,0.0,0.0,Rocky Karlage,Staci Layne Wilson,Rocky Karlage,Action,2022,1,0
3,Tyson's Run,PG,103,0.0,0.0,Kim Bass,Kim Bass,Major Dodson,Action,2022,1,0
4,X,R,0,0.0,0.0,Ti West,Ti West,Mia Goth,Action,2022,1,0
5,Operation Fortune: Ruse de guerre,0,0,0.0,0.0,Guy Ritchie,Marn Davies,Josh Hartnett,Action,2022,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
78,Aquaman and the Lost Kingdom,0,0,0.0,0.0,James Wan,Paul Norris,Temuera Morrison,Comedy,2022,1,0
79,Untitled Illumination Entertainment Project,0,0,0.0,0.0,Michael Jelenic,Shigeru Miyamoto,Seth Rogen,Drama,2022,1,0
80,I Wanna Dance with Somebody,0,0,0.0,0.0,Kasi Lemmons,Anthony McCarten,Tamara Tunie,Drama,2022,1,0


In [23]:
#heroku_df= movie.join(upcoming, on='movie_id', how='left')
#heroku_df

heroku_df = pd.concat([movie1, upcoming], axis=0)
heroku_df

,title,movie_rating,run_time,meta_score,imdb_rating,director,writer,actor,genre_name,Year,Predictions_BoxOffice,Predictions_Upcoming,label_boxoffice
movie_id,,,,,,,,,,,,,
1,The Shining,R,146.0,0.0,8.4,Stanley Kubrick,Stephen King,Jack Nicholson,Drama,1980,1.0,NaN,NaN
9,Superman II,PG,127.0,0.0,6.8,Richard Lester,Jerry Siegel,Gene Hackman,Action,1981,1.0,NaN,NaN
15,Dressed to Kill,R,104.0,0.0,7.1,Brian De Palma,Brian De Palma,Michael Caine,Crime,1980,1.0,NaN,NaN
16,Somewhere in Time,PG,103.0,0.0,7.2,Jeannot Szwarc,Richard Matheson,Christopher Reeve,Drama,1980,1.0,NaN,NaN
18,9 to 5,PG,109.0,0.0,6.9,Colin Higgins,Patricia Resnick,Jane Fonda,Comedy,1980,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,Aquaman and the Lost Kingdom,0,0.0,0.0,0.0,James Wan,Paul Norris,Temuera Morrison,Comedy,2022,NaN,1.0,0.0
79,Untitled Illumination Entertainment Project,0,0.0,0.0,0.0,Michael Jelenic,Shigeru Miyamoto,Seth Rogen,Drama,2022,NaN,1.0,0.0
80,I Wanna Dance with Somebody,0,0.0,0.0,0.0,Kasi Lemmons,Anthony McCarten,Tamara Tunie,Drama,2022,NaN,1.0,0.0


In [24]:
heroku_df['Predictions_BoxOffice'] = heroku_df['Predictions_BoxOffice'].fillna(0)
heroku_df

,title,movie_rating,run_time,meta_score,imdb_rating,director,writer,actor,genre_name,Year,Predictions_BoxOffice,Predictions_Upcoming,label_boxoffice
movie_id,,,,,,,,,,,,,
1,The Shining,R,146.0,0.0,8.4,Stanley Kubrick,Stephen King,Jack Nicholson,Drama,1980,1.0,NaN,NaN
9,Superman II,PG,127.0,0.0,6.8,Richard Lester,Jerry Siegel,Gene Hackman,Action,1981,1.0,NaN,NaN
15,Dressed to Kill,R,104.0,0.0,7.1,Brian De Palma,Brian De Palma,Michael Caine,Crime,1980,1.0,NaN,NaN
16,Somewhere in Time,PG,103.0,0.0,7.2,Jeannot Szwarc,Richard Matheson,Christopher Reeve,Drama,1980,1.0,NaN,NaN
18,9 to 5,PG,109.0,0.0,6.9,Colin Higgins,Patricia Resnick,Jane Fonda,Comedy,1980,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,Aquaman and the Lost Kingdom,0,0.0,0.0,0.0,James Wan,Paul Norris,Temuera Morrison,Comedy,2022,0.0,1.0,0.0
79,Untitled Illumination Entertainment Project,0,0.0,0.0,0.0,Michael Jelenic,Shigeru Miyamoto,Seth Rogen,Drama,2022,0.0,1.0,0.0
80,I Wanna Dance with Somebody,0,0.0,0.0,0.0,Kasi Lemmons,Anthony McCarten,Tamara Tunie,Drama,2022,0.0,1.0,0.0


In [25]:
heroku_df['Predictions_Upcoming'] = heroku_df['Predictions_Upcoming'].fillna(0)
heroku_df

,title,movie_rating,run_time,meta_score,imdb_rating,director,writer,actor,genre_name,Year,Predictions_BoxOffice,Predictions_Upcoming,label_boxoffice
movie_id,,,,,,,,,,,,,
1,The Shining,R,146.0,0.0,8.4,Stanley Kubrick,Stephen King,Jack Nicholson,Drama,1980,1.0,0.0,NaN
9,Superman II,PG,127.0,0.0,6.8,Richard Lester,Jerry Siegel,Gene Hackman,Action,1981,1.0,0.0,NaN
15,Dressed to Kill,R,104.0,0.0,7.1,Brian De Palma,Brian De Palma,Michael Caine,Crime,1980,1.0,0.0,NaN
16,Somewhere in Time,PG,103.0,0.0,7.2,Jeannot Szwarc,Richard Matheson,Christopher Reeve,Drama,1980,1.0,0.0,NaN
18,9 to 5,PG,109.0,0.0,6.9,Colin Higgins,Patricia Resnick,Jane Fonda,Comedy,1980,1.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,Aquaman and the Lost Kingdom,0,0.0,0.0,0.0,James Wan,Paul Norris,Temuera Morrison,Comedy,2022,0.0,1.0,0.0
79,Untitled Illumination Entertainment Project,0,0.0,0.0,0.0,Michael Jelenic,Shigeru Miyamoto,Seth Rogen,Drama,2022,0.0,1.0,0.0
80,I Wanna Dance with Somebody,0,0.0,0.0,0.0,Kasi Lemmons,Anthony McCarten,Tamara Tunie,Drama,2022,0.0,1.0,0.0


In [26]:
heroku_df['label_boxoffice'] = heroku_df['label_boxoffice'].fillna(0)
heroku_df

,title,movie_rating,run_time,meta_score,imdb_rating,director,writer,actor,genre_name,Year,Predictions_BoxOffice,Predictions_Upcoming,label_boxoffice
movie_id,,,,,,,,,,,,,
1,The Shining,R,146.0,0.0,8.4,Stanley Kubrick,Stephen King,Jack Nicholson,Drama,1980,1.0,0.0,0.0
9,Superman II,PG,127.0,0.0,6.8,Richard Lester,Jerry Siegel,Gene Hackman,Action,1981,1.0,0.0,0.0
15,Dressed to Kill,R,104.0,0.0,7.1,Brian De Palma,Brian De Palma,Michael Caine,Crime,1980,1.0,0.0,0.0
16,Somewhere in Time,PG,103.0,0.0,7.2,Jeannot Szwarc,Richard Matheson,Christopher Reeve,Drama,1980,1.0,0.0,0.0
18,9 to 5,PG,109.0,0.0,6.9,Colin Higgins,Patricia Resnick,Jane Fonda,Comedy,1980,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,Aquaman and the Lost Kingdom,0,0.0,0.0,0.0,James Wan,Paul Norris,Temuera Morrison,Comedy,2022,0.0,1.0,0.0
79,Untitled Illumination Entertainment Project,0,0.0,0.0,0.0,Michael Jelenic,Shigeru Miyamoto,Seth Rogen,Drama,2022,0.0,1.0,0.0
80,I Wanna Dance with Somebody,0,0.0,0.0,0.0,Kasi Lemmons,Anthony McCarten,Tamara Tunie,Drama,2022,0.0,1.0,0.0


In [27]:
# save csv
heroku_df.to_csv("heroku_df.csv", index=False)
heroku_df

,title,movie_rating,run_time,meta_score,imdb_rating,director,writer,actor,genre_name,Year,Predictions_BoxOffice,Predictions_Upcoming,label_boxoffice
movie_id,,,,,,,,,,,,,
1,The Shining,R,146.0,0.0,8.4,Stanley Kubrick,Stephen King,Jack Nicholson,Drama,1980,1.0,0.0,0.0
9,Superman II,PG,127.0,0.0,6.8,Richard Lester,Jerry Siegel,Gene Hackman,Action,1981,1.0,0.0,0.0
15,Dressed to Kill,R,104.0,0.0,7.1,Brian De Palma,Brian De Palma,Michael Caine,Crime,1980,1.0,0.0,0.0
16,Somewhere in Time,PG,103.0,0.0,7.2,Jeannot Szwarc,Richard Matheson,Christopher Reeve,Drama,1980,1.0,0.0,0.0
18,9 to 5,PG,109.0,0.0,6.9,Colin Higgins,Patricia Resnick,Jane Fonda,Comedy,1980,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,Aquaman and the Lost Kingdom,0,0.0,0.0,0.0,James Wan,Paul Norris,Temuera Morrison,Comedy,2022,0.0,1.0,0.0
79,Untitled Illumination Entertainment Project,0,0.0,0.0,0.0,Michael Jelenic,Shigeru Miyamoto,Seth Rogen,Drama,2022,0.0,1.0,0.0
80,I Wanna Dance with Somebody,0,0.0,0.0,0.0,Kasi Lemmons,Anthony McCarten,Tamara Tunie,Drama,2022,0.0,1.0,0.0


In [40]:
movie_imbd = pd.read_csv('predictions_imbdheroku.csv', index_col=0)
movie_imbd

,title,movie_rating,run_time,meta_score,box_office,director,writer,actor,genre_name,Year,Predictions_imbdRating,label_imbd_score
movie_id,,,,,,,,,,,,
1,The Shining,R,146.0,0.0,46998772.0,Stanley Kubrick,Stephen King,Jack Nicholson,Drama,1980,1,2
9,Superman II,PG,127.0,0.0,108185706.0,Richard Lester,Jerry Siegel,Gene Hackman,Action,1981,1,1
15,Dressed to Kill,R,104.0,0.0,31899000.0,Brian De Palma,Brian De Palma,Michael Caine,Crime,1980,1,2
16,Somewhere in Time,PG,103.0,0.0,9709597.0,Jeannot Szwarc,Richard Matheson,Christopher Reeve,Drama,1980,1,2
18,9 to 5,PG,109.0,0.0,103300686.0,Colin Higgins,Patricia Resnick,Jane Fonda,Comedy,1980,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
7881,We Need to Do Something,Unrated,97.0,48.0,0.0,Sean King O'Grady,Max Booth III,Vinessa Shaw,Horror,2021,1,0
7916,Uncharted,PG-13,116.0,42.0,0.0,Ruben Fleischer,Rafe Judkins,Tom Holland,Action,2022,1,1
7920,Moonfall,PG-13,130.0,41.0,13122665.0,Roland Emmerich,Spenser Cohen,Patrick Wilson,Action,2022,1,1


In [41]:
movie_imbd1 = movie_imbd.drop(['label_imbd_score'],axis=1)
movie_imbd1

,title,movie_rating,run_time,meta_score,box_office,director,writer,actor,genre_name,Year,Predictions_imbdRating
movie_id,,,,,,,,,,,
1,The Shining,R,146.0,0.0,46998772.0,Stanley Kubrick,Stephen King,Jack Nicholson,Drama,1980,1
9,Superman II,PG,127.0,0.0,108185706.0,Richard Lester,Jerry Siegel,Gene Hackman,Action,1981,1
15,Dressed to Kill,R,104.0,0.0,31899000.0,Brian De Palma,Brian De Palma,Michael Caine,Crime,1980,1
16,Somewhere in Time,PG,103.0,0.0,9709597.0,Jeannot Szwarc,Richard Matheson,Christopher Reeve,Drama,1980,1
18,9 to 5,PG,109.0,0.0,103300686.0,Colin Higgins,Patricia Resnick,Jane Fonda,Comedy,1980,1
...,...,...,...,...,...,...,...,...,...,...,...
7881,We Need to Do Something,Unrated,97.0,48.0,0.0,Sean King O'Grady,Max Booth III,Vinessa Shaw,Horror,2021,1
7916,Uncharted,PG-13,116.0,42.0,0.0,Ruben Fleischer,Rafe Judkins,Tom Holland,Action,2022,1
7920,Moonfall,PG-13,130.0,41.0,13122665.0,Roland Emmerich,Spenser Cohen,Patrick Wilson,Action,2022,1


In [42]:
upcoming_imbd = pd.read_csv('predictions_upcoming_imbdheroku.csv', index_col=0)
upcoming_imbd

,title,movie_rating,run_time,meta_score,box_office,director,writer,actor,genre_name,Year,Predictions_imbd_Upcoming,label_imbd_score
movie_id,,,,,,,,,,,,
1,A Day to Die,0,0,0.0,0.0,Wes Miller,Scott Mallace,Kevin Dillon,Action,1970,1,0
2,He Was Asking for It: A Noir Film,0,0,0.0,0.0,Rocky Karlage,Staci Layne Wilson,Rocky Karlage,Action,2022,1,0
3,Tyson's Run,PG,103,0.0,0.0,Kim Bass,Kim Bass,Major Dodson,Action,2022,1,0
4,X,R,0,0.0,0.0,Ti West,Ti West,Mia Goth,Action,2022,1,0
5,Operation Fortune: Ruse de guerre,0,0,0.0,0.0,Guy Ritchie,Marn Davies,Josh Hartnett,Action,2022,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
78,Aquaman and the Lost Kingdom,0,0,0.0,0.0,James Wan,Paul Norris,Temuera Morrison,Comedy,2022,1,0
79,Untitled Illumination Entertainment Project,0,0,0.0,0.0,Michael Jelenic,Shigeru Miyamoto,Seth Rogen,Drama,2022,1,0
80,I Wanna Dance with Somebody,0,0,0.0,0.0,Kasi Lemmons,Anthony McCarten,Tamara Tunie,Drama,2022,1,0


In [43]:
heroku_df_final = pd.concat([movie_imbd1, upcoming_imbd], axis=0)
heroku_df_final

,title,movie_rating,run_time,meta_score,box_office,director,writer,actor,genre_name,Year,Predictions_imbdRating,Predictions_imbd_Upcoming,label_imbd_score
movie_id,,,,,,,,,,,,,
1,The Shining,R,146.0,0.0,46998772.0,Stanley Kubrick,Stephen King,Jack Nicholson,Drama,1980,1.0,NaN,NaN
9,Superman II,PG,127.0,0.0,108185706.0,Richard Lester,Jerry Siegel,Gene Hackman,Action,1981,1.0,NaN,NaN
15,Dressed to Kill,R,104.0,0.0,31899000.0,Brian De Palma,Brian De Palma,Michael Caine,Crime,1980,1.0,NaN,NaN
16,Somewhere in Time,PG,103.0,0.0,9709597.0,Jeannot Szwarc,Richard Matheson,Christopher Reeve,Drama,1980,1.0,NaN,NaN
18,9 to 5,PG,109.0,0.0,103300686.0,Colin Higgins,Patricia Resnick,Jane Fonda,Comedy,1980,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,Aquaman and the Lost Kingdom,0,0.0,0.0,0.0,James Wan,Paul Norris,Temuera Morrison,Comedy,2022,NaN,1.0,0.0
79,Untitled Illumination Entertainment Project,0,0.0,0.0,0.0,Michael Jelenic,Shigeru Miyamoto,Seth Rogen,Drama,2022,NaN,1.0,0.0
80,I Wanna Dance with Somebody,0,0.0,0.0,0.0,Kasi Lemmons,Anthony McCarten,Tamara Tunie,Drama,2022,NaN,1.0,0.0


In [56]:
heroku_df_final['Predictions_imbdRating'] = heroku_df_final['Predictions_imbdRating'].fillna(0)
heroku_df_final

,title,movie_rating,run_time,meta_score,box_office,director,writer,actor,genre_name,Year,Predictions_imbdRating,Predictions_imbd_Upcoming,label_imbd_score
movie_id,,,,,,,,,,,,,
1,The Shining,R,146.0,0.0,46998772.0,Stanley Kubrick,Stephen King,Jack Nicholson,Drama,1980,1.0,NaN,NaN
9,Superman II,PG,127.0,0.0,108185706.0,Richard Lester,Jerry Siegel,Gene Hackman,Action,1981,1.0,NaN,NaN
15,Dressed to Kill,R,104.0,0.0,31899000.0,Brian De Palma,Brian De Palma,Michael Caine,Crime,1980,1.0,NaN,NaN
16,Somewhere in Time,PG,103.0,0.0,9709597.0,Jeannot Szwarc,Richard Matheson,Christopher Reeve,Drama,1980,1.0,NaN,NaN
18,9 to 5,PG,109.0,0.0,103300686.0,Colin Higgins,Patricia Resnick,Jane Fonda,Comedy,1980,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,Aquaman and the Lost Kingdom,0,0.0,0.0,0.0,James Wan,Paul Norris,Temuera Morrison,Comedy,2022,0.0,1.0,0.0
79,Untitled Illumination Entertainment Project,0,0.0,0.0,0.0,Michael Jelenic,Shigeru Miyamoto,Seth Rogen,Drama,2022,0.0,1.0,0.0
80,I Wanna Dance with Somebody,0,0.0,0.0,0.0,Kasi Lemmons,Anthony McCarten,Tamara Tunie,Drama,2022,0.0,1.0,0.0


In [57]:
heroku_df_final['Predictions_imbd_Upcoming'] = heroku_df_final['Predictions_imbd_Upcoming'].fillna(0)
heroku_df_final

,title,movie_rating,run_time,meta_score,box_office,director,writer,actor,genre_name,Year,Predictions_imbdRating,Predictions_imbd_Upcoming,label_imbd_score
movie_id,,,,,,,,,,,,,
1,The Shining,R,146.0,0.0,46998772.0,Stanley Kubrick,Stephen King,Jack Nicholson,Drama,1980,1.0,0.0,NaN
9,Superman II,PG,127.0,0.0,108185706.0,Richard Lester,Jerry Siegel,Gene Hackman,Action,1981,1.0,0.0,NaN
15,Dressed to Kill,R,104.0,0.0,31899000.0,Brian De Palma,Brian De Palma,Michael Caine,Crime,1980,1.0,0.0,NaN
16,Somewhere in Time,PG,103.0,0.0,9709597.0,Jeannot Szwarc,Richard Matheson,Christopher Reeve,Drama,1980,1.0,0.0,NaN
18,9 to 5,PG,109.0,0.0,103300686.0,Colin Higgins,Patricia Resnick,Jane Fonda,Comedy,1980,1.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,Aquaman and the Lost Kingdom,0,0.0,0.0,0.0,James Wan,Paul Norris,Temuera Morrison,Comedy,2022,0.0,1.0,0.0
79,Untitled Illumination Entertainment Project,0,0.0,0.0,0.0,Michael Jelenic,Shigeru Miyamoto,Seth Rogen,Drama,2022,0.0,1.0,0.0
80,I Wanna Dance with Somebody,0,0.0,0.0,0.0,Kasi Lemmons,Anthony McCarten,Tamara Tunie,Drama,2022,0.0,1.0,0.0


In [58]:
heroku_df_final['label_imbd_score'] = heroku_df_final['label_imbd_score'].fillna(0)
heroku_df_final

,title,movie_rating,run_time,meta_score,box_office,director,writer,actor,genre_name,Year,Predictions_imbdRating,Predictions_imbd_Upcoming,label_imbd_score
movie_id,,,,,,,,,,,,,
1,The Shining,R,146.0,0.0,46998772.0,Stanley Kubrick,Stephen King,Jack Nicholson,Drama,1980,1.0,0.0,0.0
9,Superman II,PG,127.0,0.0,108185706.0,Richard Lester,Jerry Siegel,Gene Hackman,Action,1981,1.0,0.0,0.0
15,Dressed to Kill,R,104.0,0.0,31899000.0,Brian De Palma,Brian De Palma,Michael Caine,Crime,1980,1.0,0.0,0.0
16,Somewhere in Time,PG,103.0,0.0,9709597.0,Jeannot Szwarc,Richard Matheson,Christopher Reeve,Drama,1980,1.0,0.0,0.0
18,9 to 5,PG,109.0,0.0,103300686.0,Colin Higgins,Patricia Resnick,Jane Fonda,Comedy,1980,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,Aquaman and the Lost Kingdom,0,0.0,0.0,0.0,James Wan,Paul Norris,Temuera Morrison,Comedy,2022,0.0,1.0,0.0
79,Untitled Illumination Entertainment Project,0,0.0,0.0,0.0,Michael Jelenic,Shigeru Miyamoto,Seth Rogen,Drama,2022,0.0,1.0,0.0
80,I Wanna Dance with Somebody,0,0.0,0.0,0.0,Kasi Lemmons,Anthony McCarten,Tamara Tunie,Drama,2022,0.0,1.0,0.0


In [60]:
heroku_df_final2 = pd.concat([heroku_df, heroku_df_final], axis=0)
heroku_df_final2

,title,movie_rating,run_time,meta_score,imdb_rating,director,writer,actor,genre_name,Year,Predictions_BoxOffice,Predictions_Upcoming,label_boxoffice,box_office,Predictions_imbdRating,Predictions_imbd_Upcoming,label_imbd_score
movie_id,,,,,,,,,,,,,,,,,
1,The Shining,R,146.0,0.0,8.4,Stanley Kubrick,Stephen King,Jack Nicholson,Drama,1980,1.0,0.0,0.0,NaN,NaN,NaN,NaN
9,Superman II,PG,127.0,0.0,6.8,Richard Lester,Jerry Siegel,Gene Hackman,Action,1981,1.0,0.0,0.0,NaN,NaN,NaN,NaN
15,Dressed to Kill,R,104.0,0.0,7.1,Brian De Palma,Brian De Palma,Michael Caine,Crime,1980,1.0,0.0,0.0,NaN,NaN,NaN,NaN
16,Somewhere in Time,PG,103.0,0.0,7.2,Jeannot Szwarc,Richard Matheson,Christopher Reeve,Drama,1980,1.0,0.0,0.0,NaN,NaN,NaN,NaN
18,9 to 5,PG,109.0,0.0,6.9,Colin Higgins,Patricia Resnick,Jane Fonda,Comedy,1980,1.0,0.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,Aquaman and the Lost Kingdom,0,0.0,0.0,NaN,James Wan,Paul Norris,Temuera Morrison,Comedy,2022,NaN,NaN,NaN,0.0,0.0,1.0,0.0
79,Untitled Illumination Entertainment Project,0,0.0,0.0,NaN,Michael Jelenic,Shigeru Miyamoto,Seth Rogen,Drama,2022,NaN,NaN,NaN,0.0,0.0,1.0,0.0
80,I Wanna Dance with Somebody,0,0.0,0.0,NaN,Kasi Lemmons,Anthony McCarten,Tamara Tunie,Drama,2022,NaN,NaN,NaN,0.0,0.0,1.0,0.0


In [66]:
final_df = heroku_df.merge(heroku_df_final, how = 'left')
final_df

,title,movie_rating,run_time,meta_score,imdb_rating,director,writer,actor,genre_name,Year,Predictions_BoxOffice,Predictions_Upcoming,label_boxoffice,box_office,Predictions_imbdRating,Predictions_imbd_Upcoming,label_imbd_score
0,The Shining,R,146.0,0.0,8.4,Stanley Kubrick,Stephen King,Jack Nicholson,Drama,1980,1.0,0.0,0.0,46998772.0,1.0,0.0,0.0
1,Superman II,PG,127.0,0.0,6.8,Richard Lester,Jerry Siegel,Gene Hackman,Action,1981,1.0,0.0,0.0,108185706.0,1.0,0.0,0.0
2,Dressed to Kill,R,104.0,0.0,7.1,Brian De Palma,Brian De Palma,Michael Caine,Crime,1980,1.0,0.0,0.0,31899000.0,1.0,0.0,0.0
3,Somewhere in Time,PG,103.0,0.0,7.2,Jeannot Szwarc,Richard Matheson,Christopher Reeve,Drama,1980,1.0,0.0,0.0,9709597.0,1.0,0.0,0.0
4,9 to 5,PG,109.0,0.0,6.9,Colin Higgins,Patricia Resnick,Jane Fonda,Comedy,1980,1.0,0.0,0.0,103300686.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1979,Aquaman and the Lost Kingdom,0,0.0,0.0,0.0,James Wan,Paul Norris,Temuera Morrison,Comedy,2022,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1980,Untitled Illumination Entertainment Project,0,0.0,0.0,0.0,Michael Jelenic,Shigeru Miyamoto,Seth Rogen,Drama,2022,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1981,I Wanna Dance with Somebody,0,0.0,0.0,0.0,Kasi Lemmons,Anthony McCarten,Tamara Tunie,Drama,2022,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1982,A Man Called Otto,0,0.0,0.0,0.0,Marc Forster,Fredrik Backman,Tom Hanks,Crime,1970,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [67]:
# save csv
final_df.to_csv("heroku_df_final.csv", index=False)
final_df

,title,movie_rating,run_time,meta_score,imdb_rating,director,writer,actor,genre_name,Year,Predictions_BoxOffice,Predictions_Upcoming,label_boxoffice,box_office,Predictions_imbdRating,Predictions_imbd_Upcoming,label_imbd_score
0,The Shining,R,146.0,0.0,8.4,Stanley Kubrick,Stephen King,Jack Nicholson,Drama,1980,1.0,0.0,0.0,46998772.0,1.0,0.0,0.0
1,Superman II,PG,127.0,0.0,6.8,Richard Lester,Jerry Siegel,Gene Hackman,Action,1981,1.0,0.0,0.0,108185706.0,1.0,0.0,0.0
2,Dressed to Kill,R,104.0,0.0,7.1,Brian De Palma,Brian De Palma,Michael Caine,Crime,1980,1.0,0.0,0.0,31899000.0,1.0,0.0,0.0
3,Somewhere in Time,PG,103.0,0.0,7.2,Jeannot Szwarc,Richard Matheson,Christopher Reeve,Drama,1980,1.0,0.0,0.0,9709597.0,1.0,0.0,0.0
4,9 to 5,PG,109.0,0.0,6.9,Colin Higgins,Patricia Resnick,Jane Fonda,Comedy,1980,1.0,0.0,0.0,103300686.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1979,Aquaman and the Lost Kingdom,0,0.0,0.0,0.0,James Wan,Paul Norris,Temuera Morrison,Comedy,2022,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1980,Untitled Illumination Entertainment Project,0,0.0,0.0,0.0,Michael Jelenic,Shigeru Miyamoto,Seth Rogen,Drama,2022,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1981,I Wanna Dance with Somebody,0,0.0,0.0,0.0,Kasi Lemmons,Anthony McCarten,Tamara Tunie,Drama,2022,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1982,A Man Called Otto,0,0.0,0.0,0.0,Marc Forster,Fredrik Backman,Tom Hanks,Crime,1970,0.0,1.0,0.0,0.0,0.0,1.0,0.0
